In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../../')
print(os.getcwd())
from v2.strategy.indicators.optimal_v2 import Optimal_v2
from v2.model import Trading
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.roc import RateOfChange
from v2.research.scripts.notebook_utils import notebookUtils
from v2.research.scripts.scoreboard import updateScoreboard
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from v2.strategy.strategies.atlas.atlas_v1_2 import Atlas_v1_2

/home/marshingjay/Repos/moonshot/back_testing


In [2]:
nu = notebookUtils()
model_name = "skunk"

In [3]:
def simulate_all_trades(dataset, strategy):
    coin = 'COIN'
    generic_strategy = strategy([coin])
    opened_trades = dict()
    closed_trades = dict()
    for row in dataset.itertuples():
        otkeys = list(opened_trades.keys())
        for tick in otkeys:
            opened_trades[tick][0].process(row, coin)
            if opened_trades[tick][0].calc_exit(row, coin):
                closed_trades[tick] = (row.close/opened_trades[tick][1]) - 1
                del opened_trades[tick]
        if generic_strategy.calc_entry(row, coin):
            opened_trades[row.time] = (generic_strategy, row.close)
            generic_strategy = strategy([coin])
        
    dataset['simul_profit'] = 0.0
    for k in closed_trades:
        dataset.loc[(dataset.time == k),'simul_profit']=closed_trades[k]

In [4]:
dataset, features, indicators = nu.loadData(indicators=[('SMA','close',''),('SMA','close','for_short'),('roc','SMA_for_short', 'shorter')],
                            param_spec={
                                'SMA':{'period':150}, 
                                'SMA_for_short':{'period':30}, 
                                'RateOfChangeShorter':{'period':45},
                                },
                            spans=[],)



Loading data from BTC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...


In [5]:
simulate_all_trades(dataset, Atlas_v1_2)

In [6]:
dataset

,time,high,low,close,open,volume,SMA,SMA_for_short,RateOfChange_shorter,simul_profit
549,1565896559999,10118.88,10080.84,10118.88,10082.31,79.082298,10043.037867,10043.037867,-0.006337,0.0
550,1565896619999,10121.00,10096.18,10096.89,10118.82,66.848162,10043.034333,10043.034333,-0.006256,0.0
551,1565896679999,10106.66,10093.04,10104.57,10097.55,13.553630,10043.095467,10043.095467,-0.006176,0.0
552,1565896739999,10109.00,10091.55,10099.00,10105.61,37.793028,10043.247600,10043.247600,-0.006092,0.0
553,1565896799999,10100.89,10090.77,10097.09,10099.47,22.471044,10043.474867,10043.474867,-0.006022,0.0
554,1565896859999,10118.00,10095.02,10101.25,10097.08,25.983037,10043.856533,10043.856533,-0.005961,0.0
555,1565896919999,10100.31,10080.00,10088.83,10100.31,43.124008,10044.206200,10044.206200,-0.005900,0.0
556,1565896979999,10099.00,10087.15,10095.28,10088.83,8.466335,10044.549067,10044.549067,-0.005821,0.0
557,1565897039999,10106.12,10092.51,10094.13,10096.44,14.134890,10044.876533,10044.876533,-0.005741,0.0
558,1565897099999,10099.00,10086.80,10096.30,10093.22,10.972817,10045.227933,10045.227933,-0.005639,0.0


In [9]:
len(dataset['simul_profit'].unique())

1961